In [39]:
import face_alignment
import matplotlib.pyplot as plt
import cv2
import numpy as np
import torch
import time

#fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.TWO_D, flip_input=False, device='cpu')
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.TWO_D, device='cpu', face_detector='blazeface')


face_landmarks = {
    'Jaw': {'range': (0, 17), 'color': (0, 255, 255)},        # Cyan for Jaw
    'Eyebrows': {'range': (17, 27), 'color': (255, 255, 0)},  # Yellow for Eyebrows
    'Nose': {'range': (27, 36), 'color': (255, 0, 0)},        # Blue for Nose
    'Eyes': {'range': (36, 48), 'color': (0, 0, 255)},        # Red for Eyes
    'Outer Lips': {'range': (48, 60), 'color': (255, 0, 255)},# Magenta for Outer Lips
    'Inner Lips': {'range': (60, 68), 'color': (128, 0, 128)} # Purple for Inner Lips
}

eye_threshold = 5
jaw_threshold = 5
mouth_threshold = 8

In [40]:
cap = cv2.VideoCapture('data/raw/a_numbers.mp4')
frames = []
while True:
    success, frame = cap.read()
    if not success:
        break
    
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #frame = cv2.resize(frame, None, fx=0.2, fy=0.2, interpolation=cv2.INTER_LINEAR)

    frames.append(frame)

In [37]:
# Initialize VideoWriter parameters
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

fps = 32  # Change the frames per second as needed

output_file = 'results/demo_trackers.mp4'
frame_width, frame_height = frames[0].shape[1], frames[0].shape[0]

# Create VideoWriter object
out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

num_frames = len(frames)
batch_size = 32
frame_index=0

In [38]:
for i in range(0, num_frames, batch_size):
    batch_frames = frames[i:i+batch_size]
    
    # Convert frames to the required format (assuming frames are in BGR format)
    batch = np.stack(batch_frames)
    batch = torch.Tensor(batch.transpose(0, 3, 1, 2))
    
    t_start = time.time()
    preds = fa.get_landmarks_from_batch(batch)
    print(f'BlazeFace: Execution time for batch {i+1}-{min(i+batch_size, num_frames)}: {time.time() - t_start}')
    
    # Visualize results and write to video
    for j, pred in enumerate(preds):

        frame_index = i + j
        if frame_index < num_frames:
            frame = frames[frame_index]        

            x_values = pred[:, 0].astype(np.int64)
            y_values = pred[:, 1].astype(np.int64)

            # Plot all points using scatter plot with specific colors for facial parts
            for part, part_info in face_landmarks.items():
                start, end = part_info['range']
                color = part_info['color']
    
                # Draw circles for each facial part with different colors
                for k in range(start, end):
                    cv2.circle(frame, (int(x_values[k]), int(y_values[k])), 10, color, -1)

            
            # Write the frame with landmarks to the video
            out.write(frame)

# Release the VideoWriter and close the output file
out.release()


BlazeFace: Execution time for batch 1-32: 8.04556131362915
BlazeFace: Execution time for batch 33-64: 7.819885492324829
BlazeFace: Execution time for batch 65-96: 7.685851335525513
BlazeFace: Execution time for batch 97-128: 7.8086724281311035
BlazeFace: Execution time for batch 129-160: 7.7580406665802
BlazeFace: Execution time for batch 161-192: 7.957440137863159
BlazeFace: Execution time for batch 193-224: 7.843115329742432
BlazeFace: Execution time for batch 225-256: 7.826949119567871
BlazeFace: Execution time for batch 257-288: 7.891754627227783
BlazeFace: Execution time for batch 289-315: 6.678290367126465


In [26]:
preds[0].shape

(68, 2)